# OWIDplusLIVE

Das folgende Jupyter-Notebook dokumentiert die API von OWIDplusLIVE (www.owid.de/plus/live-2021) in der Version 3.0 (ab 2025)


## Dependencies

Folgende Dependencies müssen einmalig nach dem Starten des Notebooks importiert werden. Bitte installieren Sie ggf. erforderliche Dependencies selbst.


In [1]:
from pydantic import BaseModel
import plotly.express as px
import plotly.graph_objects as go
import requests
import json
import pandas as pd

## API-Funktion und Hilfsfunktion

Dieser Code reproduziert die vollständige Funktionalität der Webanwendung OWIDplusLIVE innerhalb der lokalen Arbeitsumgebung.
Der nachfolgende Codeteil ist umfangreich und sollte einmalig vollständig ausgeführt werden. Die Ausführung benötigt lediglich wenige Sekunden.
Im Anschluss stehen sämtliche Funktionen zur Verfügung (siehe Beispiele im Abschnitt Playground unten).
Alle Funktionen sind umfassend dokumentiert, sodass über den Code-Editor jederzeit ein direkter Zugriff auf weiterführende Informationen und Parameterbeschreibungen möglich ist.


In [2]:
# Basis URL der OWIDplusLIVE-API und Layer-Definitionen.
baseUrl = "https://www.owid.de/plus/live-2021/api/v3"
# Layer Mapping
layer = {"Wortform": 0, "Lemma": 1, "POS": 2}
# Lade verfügbare Jahre.
response = requests.request("GET", f"{baseUrl}/years", headers={}, data={})
years = response.json()

# Lade Normdaten für N-Gramme (1, 2, 3).
response = requests.request("GET", f"{baseUrl}/norm", headers={}, data={})
response = response.json()
normGram = {}
normGram[1] = dict(sorted(response[0].items()))
normGram[2] = dict(sorted(response[1].items()))
normGram[3] = dict(sorted(response[2].items()))

class SearchItem(BaseModel):
    """
    Repräsentiert ein Suchobjekt für die OWIDplusLIVE-API.

    Attribute:
        Position (int): Die Position des Tokens im N-Gramm.
        Layer (int): Die Ebene (z. B. Wortform, Lemma, POS).
        Token (str): Das zu suchende Token.
    """
    Position: int
    Layer: int
    Token: str
    
    def __init__(self, Position=0, LayerName="Wortform", Token=""):
        """
        Initialisiert ein SearchItem-Objekt.

        Args:
            Position (int): Die Position des Tokens im N-Gramm.
            LayerName (str): Der Name der Ebene (z. B. "Wortform", "Lemma" oder "POS").
            Token (str): Das zu suchende Token.
        """
        super().__init__(Position=Position, Layer=layer[LayerName], Token=Token)
    
    @property
    def LayerName(self):
        """
        Liefert den Layer-Namen basierend auf dem Layer-Wert.

        Returns:
            str: Der Name der Ebene (z. B. "Wortform").
        """
        return next((key for key, value in layer.items() if value == self.Layer), None)

    @LayerName.setter
    def LayerName(self, value):
        """
        Setzt den Layer-Wert basierend auf dem Layer-Namen.

        Args:
            value (str): Der Name des Layers (z. B. "Wortform", "Lemma" oder "POS").

        Raises:
            ValueError: Wenn der Layer-Name ungültig ist.
        """
        if value in layer:
            self.Layer = layer[value]
        else:
            raise ValueError(f"Invalid LayerName: {value}")

def searchN1(year, layer1, token1, normalize=True):
    """
    Führt eine N-Gramm-Suche mit einem Token (N=1) durch.

    Args:
        year (int): Das Fokusjahr.
        layer1 (str): Der Layer auf dem gesucht wird (z. B. "Wortform", "Lemma" oder "POS").
        token1 (str): Das zu suchende Token.
        normalize (bool): Gibt an, ob die Ergebnisse normalisiert werden sollen.

    Returns:
        dict: Die Suchergebnisse, normalisiert oder roh.
    """
    res = __search(1, year, [SearchItem(Position=0, LayerName=layer1, Token=token1)])
    return __normalizeData(1, res) if normalize else res

def searchN2(year, layer1, token1, layer2, token2, normalize=True):
    """
    Führt eine N-Gramm-Suche mit zwei Tokens (N=2) durch.

    Args:
        year (int): Das Fokusjahr.
        layer1 (str): Der Layer des ersten Tokens (z. B. "Wortform", "Lemma" oder "POS").
        token1 (str): Das erste zu suchende Token.
        layer2 (str): Die Ebene des zweiten Tokens (z. B. "Wortform", "Lemma" oder "POS").
        token2 (str): Das zweite zu suchende Token.
        normalize (bool): Gibt an, ob die Ergebnisse normalisiert werden sollen.

    Returns:
        dict: Die Suchergebnisse, normalisiert oder roh.
    """
    res = __search(2, year, [SearchItem(Position=0, LayerName=layer1, Token=token1),
                              SearchItem(Position=1, LayerName=layer2, Token=token2)])
    return __normalizeData(2, res) if normalize else res

def searchN3(year, layer1, token1, layer2, token2, layer3, token3, normalize=True):
    """
    Führt eine N-Gramm-Suche mit drei Tokens (N=3) durch.

    Args:
        year (int): Das Fokusjahr.
        layer1 (str): Die Ebene des ersten Tokens (z. B. "Wortform", "Lemma" oder "POS").
        token1 (str): Das erste zu suchende Token.
        layer2 (str): Die Ebene des zweiten Tokens (z. B. "Wortform", "Lemma" oder "POS").
        token2 (str): Das zweite zu suchende Token.
        layer3 (str): Die Ebene des dritten Tokens (z. B. "Wortform", "Lemma" oder "POS").
        token3 (str): Das dritte zu suchende Token.
        normalize (bool): Gibt an, ob die Ergebnisse normalisiert werden sollen.

    Returns:
        dict: Die Suchergebnisse, normalisiert oder roh.
    """
    res = __search(3, year, [SearchItem(Position=0, LayerName=layer1, Token=token1),
                              SearchItem(Position=1, LayerName=layer2, Token=token2),
                              SearchItem(Position=2, LayerName=layer3, Token=token3)])
    return __normalizeData(3, res) if normalize else res

def searchAdvanced(year, n, items, normalize=True):
    """
    Führt eine erweiterte N-Gramm-Suche mit beliebig vielen Layern durch.

    Args:
        year (int): Das Fokusjahr.
        n (int): Die Anzahl der Tokens (N-Gramm) - N=1 bis N=3 möglich.
        items (list[SearchItem]): Die Liste der Suchobjekte.
        normalize (bool): Gibt an, ob die Ergebnisse normalisiert werden sollen.

    Returns:
        dict: Die Suchergebnisse, normalisiert oder roh.
    """
    res = __search(n, year, items)
    return __normalizeData(n, res) if normalize else res

def __getFocusYear(year):
    """
    Hilfsfunktion, um die Reihenfolge der Jahre zu bestimmen.

    Args:
        year (int): Das gewünschte Fokusjahr.

    Returns:
        list: Die Liste der Jahre in der Reihenfolge [Fokusjahr, alle anderen Jahre].
    """
    res = []
    res.append(year)
    for y in years:
        if y != year:
            res.append(y)
    return res

def __normalizeData(n, data):
    """
    Normalisiert (in pro Mio. Token) die Suchergebnisse basierend auf den Normdaten.
    Hinweis: Diese Funktion ist privat und sollte nicht direkt aufgerufen werden. Verwenden
    Sie stattdessen die `searchN1`, `searchN2`, `searchN3` oder `searchAdvanced`-Funktionen
    und setzen Sie den `normalize`-Parameter auf True.

    Args:
        n (int): Die Anzahl der Tokens (N-Gramm).
        data (dict): Die Suchergebnisse.

    Returns:
        dict: Die normalisierten Suchergebnisse.
    """
    nData = normGram[n]
    res = {}
    for key, value in data.items():
        if key not in res:
            res[key] = {}
        for date, freq in value.items():
            if date in nData and nData[date] > 0:
                res[key][date] = (freq / nData[date]) * 1000000
            else:
                res[key][date] = 0
    return res

def __search(n, year, items):
    """
    Führt die eigentliche API-Abfrage durch. Diese Funktion ist privat und sollte
    nicht direkt aufgerufen werden. Verwenden Sie stattdessen die `searchN1`, 
    `searchN2`, `searchN3` oder `searchAdvanced`-Funktionen.

    Args:
        n (int): Die Anzahl der Tokens (N-Gramm).
        year (int): Das Fokusjahr.
        items (list[SearchItem]): Die Liste der Suchobjekte.

    Returns:
        dict: Die JSON-Antwort der API.
    """
    res = None
    for y in __getFocusYear(year):    
        url = f"{baseUrl}/search"
        payload = json.dumps({
            "N": n,
            "Year": y,
            "Items":  [item.model_dump(exclude={"LayerName"}) for item in items]})
        headers = {
            'Content-Type': 'application/json'
        }
        response = requests.request("POST", url, headers=headers, data=payload)
        tmp = response.json()
        if res is None:
            res = tmp
        else:
            res = mergeYears(res, tmp)
    return res

def granulationFunc_by_date(date):
    """
    Keine Granulation, gibt das Datum unverändert tageweise zurück.
    """
    return date

def granulationFunc_by_week(date):
    """
    Granulation auf Wochenebene, gibt das Datum im Format "YYYY-WXX" zurück.
    """
    year, month, day = map(int, date.split('-'))
    week = pd.Timestamp(year, month, day).isocalendar().week
    return f"{year}-W{week:02d}"

def granulationFunc_by_month(date):
    """
    Granulation auf Monatsebene, gibt das Datum im Format "YYYY-MM" zurück.
    """
    return date[:7]

def granulationFunc_by_quarter(date):
    """
    Granulation auf Quartalsebene, gibt das Datum im Format "YYYY-QX" zurück.
    """
    return date[:4] + "-Q" + str((int(date[5:7]) - 1) // 3 + 1)

def granulationFunc_by_year(date):
    """
    Granulation auf Jahresebene, gibt das Datum im Format "YYYY" zurück.
    """
    return date[:4]

def applyGranulation(data, granulationFunc):
    """
    Wendet die angegebene Granulationsfunktion auf die Daten an.
    Args:
        data (dict): Die Suchergebnisse.
        granulationFunc (function): Die Granulationsfunktion (z. B. granulationFunc_by_week).
        
    Returns:
        dict: Die granulierteren Suchergebnisse.
    """
    res = {}
    for key, value in data.items():
        if key not in res:
            res[key] = {}
        for date, freq in value.items():
            gDate = granulationFunc(date)
            if gDate in res[key]:
                res[key][gDate] += freq
            else:
                res[key][gDate] = freq
    return res

def __safeGetVal(data, key):
    """
    Hilfsfunktion, um einen Wert aus einem Dictionary sicher abzurufen.
    Gibt 0 zurück, wenn der Schlüssel nicht vorhanden ist.
    """
    if key in data:
        return data[key]
    return 0

def applyMovingAvarage(data, avarageSize):
    """
    Wendet einen gleitenden Durchschnitt auf die Daten an.
    Args:
        data (dict): Die Suchergebnisse.
        avarageSize (int): Die Größe des gleitenden Durchschnittsfensters.
    Returns:
        dict: Die geglätteten Suchergebnisse.
    """
    if(avarageSize < 2):
        return data
    
    res = {}
    for key, value in data.items():
        if key not in res:
            res[key] = {}
        dates = sorted(value.keys())
        for i in range(len(dates)):
            start_index = max(0, i - avarageSize + 1)
            end_index = i + 1
            window_dates = dates[start_index:end_index]
            window_sum = sum(__safeGetVal(value, d) for d in window_dates)
            window_count = len(window_dates)
            res[key][dates[i]] = window_sum / window_count if window_count > 0 else 0
    return res

def mergeYears(year1, year2):
    """
    Führt die Suchergebnisse von zwei Jahren zusammen.
    Args:
        year1 (dict): Die Suchergebnisse des ersten Jahres.
        year2 (dict): Die Suchergebnisse des zweiten Jahres.
    Returns:
        dict: Die zusammengeführten Suchergebnisse.
    """
    
    for key, value in year2.items():
        if key not in year1:
            year1[key] = {}
        for date, freq in value.items():
            if date in year1[key]:
                year1[key][date] += freq
            else:
                year1[key][date] = freq
    return year1

def mergeAllResuls(data, newLabel="All"):
    """
    Führt die Suchergebnisse aller N-Gramme zusammen.
    Args:
        data (dict): Die Suchergebnisse.
        newLabel (str): Das Label für die zusammengeführten Ergebnisse.
    Returns:
        dict: Die zusammengeführten Suchergebnisse.
    """
    res = {}
    for key, value in data.items():
        for date, freq in value.items():
            if date in res:
                res[date] += freq
            else:
                res[date] = freq
    return {newLabel: res}

def createSankeyData(data, n):
    """
    Erstellt die Datenstruktur für einen Sankey-Plot aus den Suchergebnissen.
    Args:
        data (dict): Die Suchergebnisse.
        n (int): Die Anzahl der Tokens (N-Gramm), z. B. 1, 2 oder 3.
    Returns:
        dict: Die Datenstruktur für den Sankey-Plot.
    """
    if n < 1 or n > 3:
        raise ValueError("Nur N-Gramme von 1 bis 3 werden unterstützt.")
    
    labelDict = {"": 0}
    sources = []
    targets = []
    values = []
    
    for key, value in data.items():
        parts = key.split(" ")
        if len(parts) != n:
            continue
        
        for i in range(len(parts) - 1):
            if parts[i] not in labelDict:
                labelDict[parts[i]] = len(labelDict)
            if parts[i + 1] not in labelDict:
                labelDict[parts[i + 1]] = len(labelDict)
            sources.append(labelDict[parts[i]])
            targets.append(labelDict[parts[i + 1]])
            values.append(sum(value.values()))
    
    return go.Sankey(
        node=dict(
            label=list(labelDict.keys()),
        ),
        link=dict(
            source=sources,
            target=targets,
            value=values
        )
    )

def convertToDataFrame(data):
    """
    Konvertiert die Suchergebnisse in ein Pandas DataFrame.
    Args:
        data (dict): Die Suchergebnisse.
    Returns:
        pd.DataFrame: Das resultierende DataFrame.
    """
    records = []
    for key, value in data.items():
        for date, freq in value.items():
            records.append({"N-Gramm": key, "Datum": date, "Frequenz": freq})
    df = pd.DataFrame(records)
    df = df.sort_values(by=["Datum"])
    return df

## Playground

Hier finden Sie Beispiele zur Abfrage der API


### N=1


#### N=1 - Wortfrom = 'deutschland' mit Fokus auf 2020

Suche zuerst in 2020 nach der Wortform "deutschland" - gebe das Ergebnis ganz normal über die Konsole aus. Ausgegeben wird für jede Übereinstimmung (hier nur 'deutschland' - siehe auch nächstes Beispiel) sowie die absolute Frequenz pro Tag.


In [3]:
data = searchN1(2020, "Wortform", "deutschland")
print(data)

{'deutschland': {'2020-01-21': 1267.9136422855015, '2020-04-30': 2400.037282132538, '2020-12-03': 1991.0459177934263, '2020-11-06': 1588.4386582365216, '2020-08-10': 1182.2223316872528, '2020-10-22': 2124.708965762923, '2020-08-01': 1953.3793462690455, '2020-12-22': 1963.1213629671176, '2020-07-14': 1335.6379635449402, '2020-05-10': 3000.1875117194822, '2020-04-08': 2590.1671390870883, '2020-06-22': 1285.4182595880013, '2020-11-10': 1488.5094332772621, '2020-04-24': 1564.4370978215952, '2020-10-05': 1330.8714210349624, '2020-10-02': 1388.5293356240145, '2020-10-03': 1661.1595615983244, '2020-10-16': 1782.2053648951241, '2020-11-15': 1371.7672019607126, '2020-09-10': 2499.8355371357147, '2020-12-18': 1691.2842485060323, '2020-01-20': 1549.3057690943067, '2020-09-06': 2362.7870064751187, '2020-12-28': 1905.0893100138944, '2020-07-30': 1917.9320538975555, '2020-03-15': 2995.775619772119, '2020-06-02': 1629.2231178185561, '2020-09-29': 1698.4263822231371, '2020-06-29': 1300.3596490200443, 

#### N=1 - Lemma = 'sagen' mit Fokus 2024

Suche zuerst in 2024 nach dem Lemma 'sagen' - gebe alle Wortformen aus. Die Keys entsprechen den passenden Wortformen.


In [4]:
data = searchN1(2024, "Lemma", "sagen")
print(data.keys())

dict_keys(['sagte', 'gesagt', 'sagt', 'sagen', 'sagten', 'sage', 'sag', 'sagst', "sag's", "sagt's", 'saget'])


#### N=1 - Wortform = '\*sagte' mit Fokus 2023

Suche zuerst in 2023 nach allen Wortformen, die auf "sagte" enden. Nehme die TOP5 häufigsten N-Gramme und erzeuge eine Verlaufsgrafik.


In [5]:
# Suche
data = searchN1(2023, "Wortform", "*sagte")

# Konvertiere in DataFrame
df = convertToDataFrame(data)

# Filtere DataFrame nach top 10 N-Gramme
top = df.groupby("N-Gramm")["Frequenz"].sum().nlargest(5).index
df = df[df["N-Gramm"].isin(top)]

# Plot mit Seaborn
fig = px.line(df, x="Datum", y="Frequenz", color="N-Gramm", markers=True, title="Vergleich der Verlaufskurven mit Plotly")
fig.update_layout(
  xaxis_title="Datum",
  yaxis_title="Frequenz / Wert",
  yaxis_type="log",
  template="plotly_white"
)
fig.show()

#### N=1 - Wortform = '\*sagte' mit Fokus 2023 + Granulierung (in Wochen) + Glättung (4)

Suche zuerst in 2023 nach allen Wortformen, die auf "sagte" enden. Nehme die TOP5 häufigsten N-Gramme und erzeuge eine Verlaufsgrafik. Die Daten werden nach Wochen granuliert und ein gleitender Durschnitt von vier Wochen wird berechnet.


In [6]:
# Suche
data = searchN1(2023, "Wortform", "*sagte")

# Granuliere nach Wochen
data = applyGranulation(data, granulationFunc_by_week)
# Glätte mit gleitendem Durchschnitt (4 Wochen)
data = applyMovingAvarage(data, 4)

# Konvertiere in DataFrame
df = convertToDataFrame(data)

# Filtere DataFrame nach top 10 N-Gramme
top = df.groupby("N-Gramm")["Frequenz"].sum().nlargest(5).index
df = df[df["N-Gramm"].isin(top)]

# Plot mit Seaborn
fig = px.line(df, x="Datum", y="Frequenz", color="N-Gramm", markers=True, title="Vergleich der Verlaufskurven mit Plotly")
fig.update_layout(
  xaxis_title="Datum",
  yaxis_title="Frequenz / Wert",
  yaxis_type="log",
  template="plotly_white"
)
fig.show()


### N=2


#### N=2 - Wortform="angela merkel" mit Fokus 2020

Suche zuerst in 2020 nach "angela merkel" (N=2), granuliere nach Monaten und wenden einen gleitenden Durchschnitt von 4 Monaten an. Erstelle dann einen Plot.


In [7]:
# Suche
data = searchN2(2020, "Wortform", "angela", "Wortform", "merkel")

# Wenn mehrere Schreibvarianten, fasse die Ergebnisse zusammen
data = mergeAllResuls(data, newLabel="angela merkel")

# Granuliere nach Monaten
data = applyGranulation(data, granulationFunc_by_month)

# Berechne gleitenden Durchschnitt (4 Monate)
data = applyMovingAvarage(data, 4)

# Konvertiere in DataFrame
df = convertToDataFrame(data)

# Plot mit Seaborn
fig = px.line(df, x="Datum", y="Frequenz", color="N-Gramm", markers=True, title="Verlaufskurve für 'angela merkel'")
fig.update_layout(
  xaxis_title="Datum",
  yaxis_title="Frequenz / Wert",
  yaxis_type="log",
  template="plotly_white"
)
fig.show()

#### N=2 - 1. Lemma = "sprechen" / 2. POS = "APPR" - ein Sankey-Plot

Erstelle für "sprechen APPR" einen Sankey-Plot.


In [8]:
# Suche
data = searchN2(2024, "Lemma", "sprechen", "POS", "APPR")

# Erstelle Sankey-Daten
sankeyData = createSankeyData(data, 2)

# Erstelle Sankey-Plot
fig = go.Figure(sankeyData)
fig.update_layout(title_text="Sankey-Plot für 'sprechen APPR'", font_size=10)
fig.update_layout(width=600, height=1000)
fig.show()